In [1]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, transforms
import multiprocessing
from joblib import Parallel, delayed
from datetime import datetime
import tempfile
import os
from joblib import load, dump
import shutil

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),])
trainset = datasets.MNIST('./data', download=True, train=True, transform=transform)
testset = datasets.MNIST('./data', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = len(trainset))
testloader = torch.utils.data.DataLoader(testset, batch_size = len(testset))
trainset_array1 = next(iter(trainloader))[0].numpy()
testset_array1 = next(iter(testloader))[0].numpy()

trainset_labels_array1 = next(iter(trainloader))[1].numpy()
testset_labels_array1 = next(iter(testloader))[1].numpy()

trainset_array = trainset_array1.reshape(60000,784)
testset_array = testset_array1.reshape(10000,784)


In [2]:
temp_folder_test = tempfile.mkdtemp()
filename_test = os.path.join(temp_folder_test, 'test.mmap')
if os.path.exists(filename_test): os.unlink(filename_test)
_ = dump(testset_array, filename_test)
test_memmap = load(filename_test, mmap_mode='r+')

temp_folder_train = tempfile.mkdtemp()
filename_train = os.path.join(temp_folder_train, 'train.mmap')
if os.path.exists(filename_train): os.unlink(filename_train)
_ = dump(trainset_array, filename_train)
train_memmap = load(filename_train, mmap_mode='r+')

del testset_array
del trainset_array
import gc
_ = gc.collect()

test_memmap_small = test_memmap[:512]
train_memmap_small = train_memmap[:512]

test_memmap_final = np.asarray(test_memmap_small)
train_memmap_final = np.asarray(train_memmap_small)

In [3]:
# testset_array2 = testset_array[:512]
# trainset_array2 = trainset_array[:512]

In [4]:
def case3(a):
    list1 = []
    for y in test_memmap_final:
        val = sum((a-y)**2)
        list1.append(val)
    return list1

### Non Parallel

In [6]:
################
# non-parallel #
################

final_list01 = []
start = datetime.now()

for x in train_memmap_final:
    final_list01.append(case3(x))
# final_list01 = np.asarray(final_list01)
final02 = [list(x) for x in zip(*final_list01)] 
l01 = []
i=0
for x in final02:
    y = x.index(min(x))
    l01.append(trainset_labels_array1[y])
    i+=1

end = datetime.now()
time_taken = end - start
print('Time: ',time_taken) 
print(len(l01))


Time:  0:00:39.192323
512


In [ ]:
l01 = np.asarray(l01)
l01.shape

### Parallel

In [9]:
start = datetime.now()

final_list = Parallel(n_jobs=8,backend='multiprocessing',batch_size=1,max_nbytes=None)(delayed(case3)(x) for x in train_memmap_final)
# final_list = np.asarray(final_list)
final2 = [list(x) for x in zip(*final_list)] 
l1 = []
i=0
for x in final2:
    y = x.index(min(x))
    l1.append(trainset_labels_array1[y])
    i+=1

end = datetime.now()
time_taken = end - start
print('Time: ',time_taken) 
print(len(l1))


Time:  0:00:13.597085
512


In [ ]:
try:
    shutil.rmtree(temp_folder_test)
    shutil.rmtree(temp_folder_train)
except OSError:
    pass

In [ ]:
l1

In [ ]:
l2 = []
j = 0
for j in range(0,512):
    l2.append(testset_labels_array1[j])
l2= np.asarray(l2)
l2.shape

In [ ]:
j = 0
for x in range(0,512):
    if l1[x] == l2[x]:
        j += 1
j

In [ ]:
403/512

In [ ]:
dist_eg1 = final2[0]
print(np.min(dist_eg1))
minimum = np.argmin(dist_eg1)
print(trainset_labels_array1[minimum])
print(testset_labels_array1[0])